In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer

import re

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
main_df = pd.read_csv('DB/Full_Dataset.csv')

In [3]:
main_df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Dengue,fever,nausea,vomiting,rash,headache,pain_behind_the_eyes,joint_pain,muscle_pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Typhoid Fever,fever,fatigue,headache,nausea,abdominal_pain,rash,constipation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Typhoid Fever,fever,fatigue,headache,nausea,abdominal_pain,rash,diarrhea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Allergy,shivering,chills,watering_from_eyes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df = main_df.copy()

In [5]:
#df.dropna(axis=1, how='all', inplace=True)

In [6]:
df.fillna(method='ffill', inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_14160\3970806690.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


In [7]:
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Dengue,fever,nausea,vomiting,rash,headache,pain_behind_the_eyes,joint_pain,muscle_pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Typhoid Fever,fever,fatigue,headache,nausea,abdominal_pain,rash,constipation,muscle_pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Typhoid Fever,fever,fatigue,headache,nausea,abdominal_pain,rash,diarrhea,muscle_pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Allergy,continuous_sneezing,shivering,chills,watering_from_eyes,abdominal_pain,rash,diarrhea,muscle_pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Allergy,shivering,chills,watering_from_eyes,watering_from_eyes,abdominal_pain,rash,diarrhea,muscle_pain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
class CustomLabelEncoder(LabelEncoder):
    def __init__(self, start = 0):
        self.start = start
        super().__init__()

    def fit_transform(self, y):
        encoded = super().fit_transform(y)
        encoded += self.start
        return encoded

In [9]:
flasttened_series = df['Disease'].astype(str)

In [10]:
encoder = CustomLabelEncoder(start=200)

In [11]:
encoded_values = encoder.fit_transform(flasttened_series)

In [12]:
df['Disease'] = encoded_values

In [13]:
mapping_data = {'label_encoder' : encoder}

In [14]:
label_mapping = {k : v for k, v in zip(mapping_data['label_encoder'].classes_, range(200, 200 + len(mapping_data['label_encoder'].classes_)))}

In [15]:
df = main_df.copy()

In [16]:
df['All Symptoms'] = df.apply(lambda row: ','.join(row.dropna()), axis=1)
df['All Symptoms'] = df['All Symptoms'].apply(lambda x: ','.join(sorted(set(x.split(','))) if x else ''))

In [17]:
stay_cols= ['Disease', 'All Symptoms']

In [18]:
df = df[stay_cols]

In [19]:
df.head()

,Disease,All Symptoms
0,Dengue,"Dengue,fever,headache,joint_pain,muscle_pain,n..."
1,Typhoid Fever,"Typhoid Fever,abdominal_pain,constipation,fati..."
2,Typhoid Fever,"Typhoid Fever,abdominal_pain,diarrhea,fatigue,..."
3,Allergy,"chills, continuous_sneezing, shivering, water..."
4,Allergy,"chills, shivering, watering_from_eyes,Allergy"


In [20]:
def strip_to_basic_tokens(text):
    text = re.sub(r'[_\s]+', ' ', text)

    tokens = [token.strip().lower() for token in text.split(',')]
    return tokens

In [21]:
df['Basic Tokens'] = df['All Symptoms'].apply(strip_to_basic_tokens)
df['Basic Tokens'] = df['Basic Tokens'].apply(lambda x: ', '.join(x))

In [22]:
df.head()

,Disease,All Symptoms,Basic Tokens
0,Dengue,"Dengue,fever,headache,joint_pain,muscle_pain,n...","dengue, fever, headache, joint pain, muscle pa..."
1,Typhoid Fever,"Typhoid Fever,abdominal_pain,constipation,fati...","typhoid fever, abdominal pain, constipation, f..."
2,Typhoid Fever,"Typhoid Fever,abdominal_pain,diarrhea,fatigue,...","typhoid fever, abdominal pain, diarrhea, fatig..."
3,Allergy,"chills, continuous_sneezing, shivering, water...","chills, continuous sneezing, shivering, wateri..."
4,Allergy,"chills, shivering, watering_from_eyes,Allergy","chills, shivering, watering from eyes, allergy"


In [23]:
df = df.drop(['All Symptoms'], axis=1)

In [24]:
df.head()

,Disease,Basic Tokens
0,Dengue,"dengue, fever, headache, joint pain, muscle pa..."
1,Typhoid Fever,"typhoid fever, abdominal pain, constipation, f..."
2,Typhoid Fever,"typhoid fever, abdominal pain, diarrhea, fatig..."
3,Allergy,"chills, continuous sneezing, shivering, wateri..."
4,Allergy,"chills, shivering, watering from eyes, allergy"


In [25]:
dfE = df.copy()

In [26]:
dfE['Basic Tokens'] = dfE['Basic Tokens'].apply(lambda x: x.split(', '))

In [27]:
mlb = MultiLabelBinarizer()

In [28]:
one_hot_encoded = pd.DataFrame(mlb.fit_transform(dfE['Basic Tokens']), columns=mlb.classes_, index=df.index)

In [29]:
df_encoded = pd.concat([dfE, one_hot_encoded], axis=1)

In [30]:
df_encoded = df_encoded.drop(columns=['Basic Tokens'])
df_encoded.head()

,Disease,abdominal pain,acidity,allergy,back pain,blurred and distorted vision,breathlessness,bronchial asthma,burning micturition,cervical spondylosis,...,swelled lymph nodes,throat irritation,typhoid fever,visual disturbances,vomiting,watering from eyes,weakness in limbs,weight loss,yellowing of eyes,yellowish skin
0,Dengue,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,Typhoid Fever,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Typhoid Fever,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Allergy,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,Allergy,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [31]:
model_features = df_encoded.columns.to_list()

In [32]:
model_features = df_encoded.columns.to_list()

In [33]:
model_features.remove("Disease")

In [34]:
X = df_encoded[model_features]
y = df_encoded["Disease"]

In [35]:
y_encoded = pd.get_dummies(y)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)
X_eval, X_test, y_eval, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [37]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=1, class_weight='balanced')

In [38]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_jobs=1, random_state=42)

In [39]:
y_pred = rf_model.predict(X_eval)

In [40]:
y_pred_labels = np.argmax(y_pred, axis=1)
y_eval_labels = np.argmax(y_eval.values, axis=1)

In [41]:
accuracy = accuracy_score(y_eval_labels, y_pred_labels)

In [42]:
print(f"Validation Acccuracy:  {accuracy}")

Validation Acccuracy:  1.0


In [43]:
y_test_pred = rf_model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.9333333333333333


In [44]:
def predict_disease(symptoms, rf_model, mlb, encoder, df_encoded):
    # Step 1: Preprocess the symptoms
    def strip_to_basic_tokens(text):
        # Remove double spaces and underscores, then split by commas and lowercase the tokens
        text = re.sub(r'[_\s]+', ' ', text)
        tokens = [token.strip().lower() for token in text.split(',')]
        return tokens
    
    # Apply the tokenization process to the sample symptoms
    basic_tokens = strip_to_basic_tokens(symptoms)

    # Step 2: One-hot encode the tokens using the MultiLabelBinarizer (mlb) used during training
    one_hot_encoded_sample = mlb.transform([basic_tokens])
    
    # Create a DataFrame from the one-hot encoded sample
    one_hot_df = pd.DataFrame(one_hot_encoded_sample, columns=mlb.classes_)
    
    # Ensure that all columns (features) from training data are present (fill with 0 for missing columns)
    missing_columns = set(df_encoded.columns) - set(one_hot_df.columns)
    for col in missing_columns:
        one_hot_df[col] = 0
    
    # Reorder columns to match the original training DataFrame's order
    one_hot_df = one_hot_df[df_encoded.columns.difference(['Disease'])]

    # Step 3: Make prediction
    y_pred = rf_model.predict(one_hot_df)

    # Check if all values in `y_pred` are `False`
    if not y_pred.any():
        return "No_Matching"
    
    # Step 4: Decode the prediction (back to the disease label)
    predicted_class_index = np.argmax(y_pred)
    predicted_disease = encoder.inverse_transform([predicted_class_index])[0]
    
    return predicted_disease

# Example usage:
sample_symptoms = "continuous_sneezing, shivering, chills, watering_from_eyes"
#sample_symptoms = "fatigue, weight_loss, restlessness, lethargy, irregular_sugar_level, blurred_and_distorted_vision, obesity, "
#sample_symptoms = "1, 2, 3, 4"
#sample_symptoms = "continuous_sneezing, chills, fatigue, cough,fever, headache, swelled_lymph_nodes, malaise, phlegm, throat_irritation, redness_of_eyes, sinus_pressure, runny_nose, congestion, chest_pain, loss_of_smell, muscle_pain"
#sample_symptoms = "vomiting, sunken_eyes, dehydration"

predicted_disease = predict_disease(sample_symptoms, rf_model, mlb, encoder, df_encoded)

print(f"The predicted disease for the symptoms '{sample_symptoms}' is: {predicted_disease}")


The predicted disease for the symptoms 'continuous_sneezing, shivering, chills, watering_from_eyes' is: Allergy


In [45]:
import pickle

# Save the Random Forest model
with open("Models/rf_model.pkl", "wb") as model_file:
    pickle.dump(rf_model, model_file)

# Save the label encoder
with open("Models/label_encoder.pkl", "wb") as le_file:
    pickle.dump(encoder, le_file)


# Save the MultiLabelBinarizer
with open("Models/mlb.pkl", "wb") as mlb_file:
    pickle.dump(mlb, mlb_file)

print("Model, Label Encoder, and MultiLabelBinarizer saved successfully!")


Model, Label Encoder, and MultiLabelBinarizer saved successfully!
